### Basic ISC



#### Import modules

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os, nilearn, pickle, glob,shutil
from nilearn import plotting, input_data, datasets
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import multipletests
from utils import *
import warnings

warnings.simplefilter("ignore")

#### Load timeseries, run basic ISC (static)

In [2]:
twin_df = pd.read_csv('../data/00_dataset_info/twin_df.csv')
sub_list = list(np.hstack((twin_df['twin_A_id'].values, twin_df['twin_B_id'].values)))
len(sub_list)

200

In [ ]:
# done only for the 1st time, later just load the data
#'''
ts_data = np.zeros( ( 380, 124045, len(sub_list) ) ) #380 is timepoints, #124045 is in-mask voxels

for curr_sub in range(len(sub_list)):
    #print(curr_sub)
    curr_file = '/Volumes/Backup Plus/twins_partly_cloudy_extracted/' + sub_list[curr_sub] + '_LTR_Z_THP001_LP012_voxelwise.npy'
    print(curr_file)
    
    ts_data[ :, :, curr_sub] = np.load(curr_file)
    
print(ts_data.shape)

np.save('/Volumes/Backup Plus/twins_partly_cloudy_extracted/ts_array_200subs_wholebrain.npy', ts_data)
#'''
print(' ')

/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0001_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0003_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0004_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0005_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0006_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0007_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0010_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0013_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0015_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0016_LTR_Z_THP001_LP012_voxelwise.npy
/Volumes/Backup Plus/twins_partly_cloudy_extracted/sub-0017_LTR_Z_THP0

In [ ]:
# load timeseries
ts_array = np.load('/Volumes/Backup Plus/twins_partly_cloudy_extracted/ts_array_200subs_wholebrain.npy')
#ts_data = np.swapaxes(ts_array, 0, 2)
#ts_data = np.swapaxes(ts_data, 0, 1)
ts_data.shape

In [ ]:
# submit to ISC analysis

#data input shape should be 380 * 124045 * 200 

# takes about 15-20 minutes on normal machine
ISC, p, distribution = partly_phaseshift_isc(ts_data, 
                                           pairwise= False,
                                           summary_statistic = np.mean,
                                           n_shifts = 1000, #can reduce to speed up, better increase to 1000
                                           return_distribution=True)

ISC[np.isnan(ISC)] = 0

# Stats correction (uncomment if you computed enough n_shifts)
alpha                           = .05
q                               = multipletests(p[0, :], method='fdr_by')[1]
corrected                       = q[np.newaxis, :] < alpha
#ISC[corrected == False]         = 0

isc_result_dict                 = {}
isc_result_dict['ISC']          = ISC
isc_result_dict['p']            = p
isc_result_dict['distribution'] = distribution


with open('/Volumes/Backup Plus/twins_partly_cloudy_extracted/isc_result_dict.pkl', 'wb') as f:
    pickle.dump(isc_result_dict, f)

#### Plot ISC results

In [ ]:
from nilearn.maskers import NiftiMasker
brain_masker = NiftiMasker( #mask_img = mask_path,
                            detrend     = True, 
                            standardize = True,
                            high_pass   = 0.01, 
                            t_r         = 0.8, 
                            low_pass    = 0.12,
                            mask_strategy = 'whole-brain-template',
                          );
brain_masker.fit('/Volumes/Backup Plus/twins_fmriprep/sub-0001/ses-02/func/sub-0001_ses-02_task-partlycloudy_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')



curr_isc = isc_result_dict['ISC']  #np.reshape(ISC, (293,1)).T
curr_img = brain_masker.inverse_transform(curr_isc);
plt.figure(figsize = (20,15));
nilearn.plotting.plot_stat_map(nilearn.image.index_img(curr_img,0), 
                                   threshold      = 0.05, 
                                   vmax           = 0.50, 
                                   draw_cross     = True,
                                   display_mode   = 'z',
                                   cut_coords     = [ -20, -10, 0, 10, 20,  30, 40, 50],
                                   symmetric_cbar = True,
                                   annotate       = True,);

plt.show();